# Estadísticos Suficientes y Ancilares en Regresión Logística

Demostración clásica con el dataset *Breast Cancer* de scikit‑learn.

- **Regresión logística** (`statsmodels`) usa todo el vector $T=X^\top Y$ (estadístico suficiente).
- **Prueba exacta de Fisher** condiciona en los totales (estadístico ancilar para el intercepto) y contrasta la covariable binaria.


In [102]:
import numpy as np, pandas as pd
from sklearn.datasets import load_breast_cancer
import statsmodels.api as sm
from scipy.stats import fisher_exact


In [103]:
# 1. Cargar datos y crear covariable binaria (large = 1 si mean radius > mediana)
data = load_breast_cancer(as_frame=True)
df   = data.frame.copy()
df['large'] = (df['mean radius'] > df['mean radius'].median()).astype(int)
y  = df['target']           # 0 = maligno, 1 = benigno
X  = sm.add_constant(df['large'])
df.head()


mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760          0.3001              0.14710         0.2419   
1           0.07864          0.0869              0.07017         0.1812   
2           0.15990          0.1974              0.12790         0.2069   
3           0.28390          0.2414              0.10520         0.2597   
4           0.13280          0.1980              0.10430         0.1809   

   mean fractal dimension  ...  worst perimeter  worst area  worst smoothness  \
0                 0.07871  ...           184.60      2019.0            0.1622   
1                 0.05667  ...           158.80      1956.0            0.1238   
2                 0.05999  ...           152.50      1709.0            0.1444   
3                 0.09744  ...            98.87       567.7            0.2098   
4                 0.05883  ...           152.20      1575.0            0.1374   

   worst compactness  worst concavity  worst concave points  worst symmetry  \
0             0.6656           0.7119                0.2654          0.4601   
1             0.1866           0.2416                0.1860          0.2750   
2             0.4245           0.4504                0.2430          0.3613   
3             0.8663           0.6869                0.2575          0.6638   
4             0.2050           0.4000                0.1625          0.2364   

   worst fractal dimension  target  large  
0                  0.11890       0      1  
1                  0.08902       0      1  
2                  0.08758       0      1  
3                  0.17300       0      0  
4                  0.07678       0      1  

[5 rows x 32 columns]

In [104]:
# 2. Regresión logística (familia exponencial / estadístico suficiente completo T = X'Y)
model = sm.Logit(y, X)
res   = model.fit(disp=0)
print(res.summary())


                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                  569
Model:                          Logit   Df Residuals:                      567
Method:                           MLE   Df Model:                            1
Date:                Fri, 09 May 2025   Pseudo R-squ.:                  0.3586
Time:                        20:50:26   Log-Likelihood:                -241.00
converged:                       True   LL-Null:                       -375.72
Covariance Type:            nonrobust   LLR p-value:                 1.496e-60
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.7578      0.250     11.026      0.000       2.268       3.248
large         -3.5421      0.281    -12.609      0.000      -4.093      -2.992


In [105]:
# 3. Tabla 2x2 y prueba exacta condicional (Fisher) que condiciona en los totales
table = pd.crosstab(df['large'], df['target'])
table.index = ['small radius (0)', 'large radius (1)']
table.columns = ['malignant (0)', 'benign (1)']
display(table)

oddsratio, p_val = fisher_exact(table.values)
print(f"Odds ratio condicional = {oddsratio:.2f}")
print(f"Exact Fisher p-value  = {p_val:.4e}")


malignant (0)  benign (1)
small radius (0)             17         268
large radius (1)            195          89

Odds ratio condicional = 0.03
Exact Fisher p-value  = 5.0590e-60


### 📘 Aplicación de la suficiencia y condicionalidad en regresión logística

Este análisis compara dos enfoques clásicos para evaluar el efecto de una covariable binaria (`large`) sobre 
la respuesta (`target`) en un contexto binario, aplicando los principios fundamentales de **suficiencia** y **condicionalidad**.

---

#### 1. Regresión logística y estadístico suficiente $T = X^\top Y$

La regresión logística modela la probabilidad de éxito como:

$$
\mathbb{P}(Y_i = 1 \mid x_i) = \sigma(x_i^\top \beta),
$$

donde $\sigma(z) = 1 / (1 + e^{-z})$. En este modelo, con diseño fijo $X$, la **estadística suficiente** para $\beta$ es:

$$
T(Y) = X^\top Y.
$$

Esto implica que toda la información relevante sobre los parámetros está contenida en $T$. El estimador de máxima verosimilitud (MLE), así como las pruebas Wald y los intervalos construidos a partir del modelo logístico, dependen exclusivamente de esta estadística suficiente.

---

#### 2. Prueba exacta de Fisher: condicionar en parte de $T$

La **prueba exacta de Fisher** se aplica sobre una tabla de contingencia 2×2:

|           | Y = 0 (maligno) | Y = 1 (benigno) | Total |
| --------- | --------------- | --------------- | ----- |
| large = 0 | a               | b               | a + b |
| large = 1 | c               | d               | c + d |
| Total     | a + c           | b + d           | n     |

Esta prueba **condiciona en los totales marginales**, en particular:

* el número total de casos malignos y benignos (i.e., $T_1 = \sum Y_i$), y
* el número de individuos en cada nivel de la covariable `large`.

##### 📌 Hipótesis del test de Fisher:

* **$H_0$:** La distribución de $Y$ es **independiente** de `large` (no hay asociación entre la respuesta y la covariable).
* **$H_1$:** Existe una **asociación** entre `large` y la probabilidad de que el tumor sea benigno o maligno.

##### 🔍 Interpretación:

El test evalúa, dado el número total de éxitos $\sum Y_i = t_1$, la probabilidad de obtener una tabla tan extrema como la observada (o más) bajo la hipótesis nula. Se basa en la distribución **hipergeométrica**, al fijar los márgenes de la tabla y permutar aleatoriamente las etiquetas $Y$.

Desde el punto de vista del modelo logístico, **esto equivale a eliminar el intercepto** (fijando su estadístico suficiente) y realizar un contraste exacto sobre el efecto de `large`. Es decir, se trata de una **regresión logística condicional** con intercepto eliminado por condicionamiento.

---

#### 3. Comparación de enfoques

| Enfoque               | Usa toda la información | Condiciona en parte de $T$ | Estima efecto  | ¿Es exacto?              |
| --------------------- | ----------------------- | -------------------------- | -------------- | ------------------------ |
| Regresión logística   | Sí ($T = X^\top Y$)     | No                         | Sí             | No (asintótico)          |
| Test exacto de Fisher | No (solo en $T_1$)      | Sí (en $T_1$)              | No (sólo test) | Sí (distribución exacta) |

---

### ✅ Conclusión

Este análisis muestra cómo:

* La **regresión logística** se basa en la estadística suficiente completa $T = X^\top Y$, lo que garantiza eficiencia bajo el modelo.
* La **prueba de Fisher** condiciona sobre parte de $T$ para eliminar el efecto del intercepto y realizar inferencia exacta sobre la covariable de interés.

Ambos enfoques están sustentados por los principios clásicos de la inferencia estadística:

* **Suficiencia**: usar solo la información relevante para el parámetro.
* **Condicionalidad**: realizar inferencia condicionando en lo que no depende del parámetro de interés (por ejemplo, totales fijos).

Esta complementariedad permite a los científicos de datos aplicar inferencias robustas tanto en contextos con grandes muestras (regresión logística) como en situaciones con celdas pequeñas o diseños experimentales fijos (Fisher exacto).
